In [ ]:
# Import Required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

## <font style="color:rgb(134,19,348)"> Initialize the DNN module </font>
To use Models in ONNX format, you just have to use **`cv2.dnn.readNetFromONNX(model)`** and pass the model inside the function.

In [ ]:
model = '../input/model1/super_resolution.onnx'
net = cv2.dnn.readNetFromONNX(model)


##  <font style="color:rgb(134,19,348)">Read Image</font> 

In [ ]:
# Read image
image = cv2.imread("../input/dipper/teenager.jpg")

# Display image
plt.figure(figsize=[10,10])
plt.imshow(image[:,:,::-1]);plt.axis('off');

##  <font style="color:rgb(134,19,348)"> Pre-processing the image 
</font>
In the this step we are going to Resize our image to (224x224) and convert it to YCbCr Color format. This format has following color components:

- **Y:**  This is like the grayscale intensity channel, it holds all the image structure.
- **Cb:** This is the blue-difference channel. 
- **Cr:** This is the red difference channel. 

After this we some formating of the Y channel and then finally normalize it by dividing with 255.0.

In [ ]:
# Prepare the Image before fed into the network

# Creating Copy of Image
img_copy = image.copy()

# Resize the image into Required Size
img_copy = cv2.resize(img_copy, (224,224), cv2.INTER_CUBIC)

# Convert image into YcbCr
image_YCbCr = cv2.cvtColor(img_copy, cv2.COLOR_BGR2YCrCb)

# Split Y,Cb, and Cr channel 
image_Y, image_Cb, image_Cr = cv2.split(image_YCbCr)

# Covert Y channel into a numpy arrary
img_ndarray = np.asarray(image_Y)

# Reshape the image to (1,1,224,224) 
reshaped_image = img_ndarray.reshape(1,1,224,224)

# Convert to float32 and as a normalization step divide the image by 255.0
blob = reshaped_image.astype(np.float32) / 255.0

print('initial Shape of Y: {}, Shape After Preprocessing: {}'.format(image_Y.shape, blob.shape))

### <font style="color:rgb(134,19,348)"> Input the Blob Image to the Network  </font>

In [ ]:
# Passing the blob as input through the network 
net.setInput(blob)

##  <font style="color:rgb(134,19,348)">Forward Pass</font> 


In [ ]:
%%time
Output = net.forward()

In [ ]:
# Our iamge has been resized to 672, a 3x increase.
print(Output.shape)


##  <font style="color:rgb(134,19,348)"> Post-processing the image </font> 
We got a 4D matrix as the output, now in the post-processing step you undo the steps you did in preprocessing.

In [ ]:
# Reshape the output and get rid of those extra dimensions
reshaped_output = Output.reshape(672,672)

# Get the image back to the range 0-255 from 0-1
reshaped_output = reshaped_output * 255

# Clip the values so the output is it between 0-255
Final_Output = np.clip(reshaped_output, 0, 255)

# Resize the Cb & Cr channel according to the output dimension
resized_Cb = cv2.resize(image_Cb,(672,672),cv2.INTER_CUBIC)
resized_Cr = cv2.resize(image_Cr,(672,672),cv2.INTER_CUBIC)

# Merge all 3 channels together 
Final_Img = cv2.merge((Final_Output.astype('uint8'), resized_Cb, resized_Cr))

# Covert back to BGR channel
Final_Img = cv2.cvtColor(Final_Img,cv2.COLOR_YCR_CB2BGR)
print(Final_Img.shape)

##  <font style="color:rgb(134,19,348)">Display Final Result 
</font>

In [ ]:
# This is how the image would look with Bicubic interpolation.
image_copy = cv2.resize(image,(672,672), cv2.INTER_CUBIC)

# Display the Bicubic Image and Super Resolution Image
plt.figure(figsize=[20,20])
plt.subplot(1,2,1);plt.imshow(image_copy[:,:,::-1]);plt.title("Bicubic Interpolation");plt.axis("off");
plt.subplot(1,2,2);plt.imshow(Final_Img[:,:,::-1]);plt.title("Super Resolution");plt.axis("off");

### <font style="color:rgb(134,19,348)"> Initialization Function </font>
This method will be run once and it will initialize the network with the required files.

In [ ]:
def init_superres(model="Model/super_resolution.onnx"):
    # Set global variables
    global net
    
    # Initialize the DNN module
    net = cv2.dnn.readNetFromONNX(model)

### <font style="color:rgb(134,19,348)">  Main Function </font>

In [ ]:
def super_res(image,returndata=False):
    
    # Create a Copy of Image
    img_copy = image.copy()

    # Resize the image into Required Size
    img_copy = cv2.resize(img_copy, (224,224), cv2.INTER_CUBIC)

    # Convert image into YcbCr
    image_YCbCr = cv2.cvtColor(img_copy, cv2.COLOR_BGR2YCrCb)

    # Split Y,Cb, and Cr channel 
    image_Y, image_Cb, image_Cr = cv2.split(image_YCbCr)

    # Covert Y channel into a numpy arrary
    img_ndarray = np.asarray(image_Y)

    # Reshape the image to (1,1,224,224) 
    reshaped_image = img_ndarray.reshape(1,1,224,224)

    # Convert to float32 and as a normalization step divide the image by 255.0
    blob = reshaped_image.astype(np.float32) / 255.0
    
    # Passing the blob as input through the network 
    net.setInput(blob)
    
    # Forward Pass
    Output = net.forward()
    
    # Reshape the output and get rid of those extra dimensions
    reshaped_output = Output.reshape(672,672)

    # Get the image back to the range 0-255 from 0-1
    reshaped_output = reshaped_output * 255

    # Clip the values so the output is it between 0-255
    Final_Output = np.clip(reshaped_output, 0, 255)

    # Resize the Cb & Cr channel according to the output dimension
    resized_Cb = cv2.resize(image_Cb,(672,672),cv2.INTER_CUBIC)
    resized_Cr = cv2.resize(image_Cr,(672,672),cv2.INTER_CUBIC)

    # Merge all 3 channels together 
    Final_Img = cv2.merge((Final_Output.astype('uint8'), resized_Cb, resized_Cr))

    # Covert back to BGR channel
    Final_Img = cv2.cvtColor(Final_Img,cv2.COLOR_YCR_CB2BGR)
    
    if  returndata:
        return Final_Img
    
    else:
        # This is how the image would look with Bicubic interpolation.
        image_copy = cv2.resize(image,(672,672), cv2.INTER_CUBIC)
        
        # Display Image
        plt.figure(figsize=[20,20])
        plt.subplot(1,2,1);plt.imshow(image_copy[:,:,::-1]);plt.title("Bicubic Interpolation");plt.axis("off");
        plt.subplot(1,2,2);plt.imshow(Final_Img[:,:,::-1]);plt.title("Super Resolution");plt.axis("off");       

**Initialize the Super Resolution**

In [ ]:
init_superres()

***Calling main function  temporary examples fahim ye add nhi ki hui mene***

In [ ]:
image = cv2.imread("../input/dipper/teenager.jpg")
super_res(image)

In [ ]:
image = cv2.imread("../input/dipper/oldman.jpg")
super_res(image)

In [ ]:
image = cv2.imread("../input/dipper/youngman.jpg")
super_res(image)

In [ ]:
image = cv2.imread("../input/dipper/youngman2.jpg")
super_res(image)